<a href="https://colab.research.google.com/github/gehirn-nerv/Centrifugal-force/blob/main/Restroom_freedom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# 安装必要的库
!pip install requests ipywidgets

import requests
import json
import ipywidgets as widgets
from IPython.display import display, clear_output

# 设置 API 密钥和端点
api_key = "sk-8a28c7412f3e4f67b7a28b1cad45774c"
api_url = "https://api.deepseek.com/chat/completions"

# 聊天历史记录（多轮对话）
chat_history = [
    {"role": "system", "content": "You are a helpful assistant."}
]

# 创建输出区域
output_area = widgets.Output(layout=widgets.Layout(border="1px solid black", width="600px"))

# 创建输入框、模式切换按钮和提交按钮（先不显示输入框）
toggle_button = widgets.ToggleButton(value=False, description="开启深度思考模式")
submit_button = widgets.Button(description="发送请求", button_style="success")

# 切换模式按钮事件
def on_toggle_change(change):
    toggle_button.description = "关闭深度思考模式" if change["new"] else "开启深度思考模式"

toggle_button.observe(on_toggle_change, names="value")

# 提交按钮事件
def on_submit_clicked(b):
    # 先隐藏输入框，避免用户在 AI 回复前输入新的问题
    submit_button.layout.display = "none"
    input_text.layout.display = "none"

    with output_area:
        text = input_text.value.strip()
        if not text:
            print("请输入内容")
            submit_button.layout.display = "inline-block"
            input_text.layout.display = "block"
            return

        # 追加用户问题到聊天记录
        chat_history.append({"role": "user", "content": text})

        # 根据模式选择模型
        model = "deepseek-reasoner" if toggle_button.value else "deepseek-chat"

        # 显示用户的问题
        clear_output(wait=True)
        print("聊天记录：")
        for msg in chat_history:
            if msg["role"] == "user":
                print(f"\n**你**：{msg['content']}")
            elif msg["role"] == "assistant":
                print(f"\n**DeepSeek**：{msg['content']}")

        print(f"\n当前模式: {'深度思考' if toggle_button.value else '普通模式'}")
        print("请求中，请稍候...\n")

        headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}
        data = {"model": model, "messages": chat_history, "max_tokens": 300}

        try:
            response = requests.post(api_url, headers=headers, data=json.dumps(data))

            if response.status_code == 200:
                result = response.json()
                reply = result.get("choices", [{}])[0].get("message", {}).get("content", "无返回内容")

                # 追加 AI 回复到聊天记录
                chat_history.append({"role": "assistant", "content": reply})

                # 重新显示完整聊天记录
                clear_output(wait=True)
                print("聊天记录：")
                for msg in chat_history:
                    if msg["role"] == "user":
                        print(f"\n**你**：{msg['content']}")
                    elif msg["role"] == "assistant":
                        print(f"\n**DeepSeek**：{msg['content']}")

            else:
                print(f"请求失败，状态码：{response.status_code}")
                print("错误信息：", response.text)

        except Exception as e:
            print("请求发生异常：", e)

        # 清空输入框，并重新显示它
        input_text.value = ""
        input_text.layout.display = "block"
        submit_button.layout.display = "inline-block"

# 创建输入框（初始隐藏，等待 AI 回答完后才显示）
input_text = widgets.Textarea(layout=widgets.Layout(width='600px', height='100px'))
submit_button.on_click(on_submit_clicked)

# 显示所有组件
display(output_area, input_text, toggle_button, submit_button)

Output(layout=Layout(border='1px solid black', width='600px'))

Textarea(value='', layout=Layout(height='100px', width='600px'))

ToggleButton(value=False, description='开启深度思考模式')

Button(button_style='success', description='发送请求', style=ButtonStyle())